In [55]:
from Unet import *
from data import *
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [8]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'dataset/train','image','label',data_gen_args,save_to_dir = None)
validGene= trainGenerator(2,'dataset/validation','image','label',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
hist = model.fit_generator(myGene, steps_per_epoch=1000, epochs=7, callbacks=[model_checkpoint], 
                     validation_data=validGene, validation_steps=3)

Epoch 1/7
Found 3 images belonging to 1 classes.
Found 24 images belonging to 1 classes.
Found 3 images belonging to 1 classes.
Found 24 images belonging to 1 classes.
1000/1000 [==============================] - 934s 934ms/step - loss: 0.2365 - dice_coef: 0.9057 - val_loss: 1.0713 - val_dice_coef: 0.8620

Epoch 00001: loss improved from inf to 0.23653, saving model to unet_membrane.hdf5
Epoch 2/7
1000/1000 [==============================] - 927s 927ms/step - loss: 0.1723 - dice_coef: 0.9310 - val_loss: 1.3520 - val_dice_coef: 0.8572

Epoch 00002: loss improved from 0.23653 to 0.17229, saving model to unet_membrane.hdf5
Epoch 3/7
1000/1000 [==============================] - 926s 926ms/step - loss: 0.1546 - dice_coef: 0.9379 - val_loss: 2.0897 - val_dice_coef: 0.8512

Epoch 00003: loss improved from 0.17229 to 0.15462, saving model to unet_membrane.hdf5
Epoch 4/7
1000/1000 [==============================] - 926s 926ms/step - loss: 0.1454 - dice_coef: 0.9416 - val_loss: 0.2469 - val_dice

### Train with npy file

In [9]:
#imgs_train,imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### test your model and save predicted results

In [21]:
testGene = testGenerator('dataset/test/image')
model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.predict_generator(testGene,3,verbose=1)
saveResult("dataset/test/results",results)

3/3 [==============================] - 1s 444ms/step


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [67]:
import matplotlib.pyplot as plt
img = io.imread('./dataset/test/label/0.png')
img = img.astype(np.float32)
img1 = trans.resize(img,[256,256])

In [68]:
img = io.imread('./dataset/test/results/0_predict.png')
img=img.astype(np.float32)

In [69]:
mean1=dice_coef(img1/255, img/255)
print("Mean Dice Coeff : ",mean1)

Mean Dice Coeff :  tf.Tensor(0.94498014, shape=(), dtype=float32)


In [73]:
img = io.imread('./dataset/test/label/1.png')
img = img.astype(np.float32)
img2 = trans.resize(img,[256,256])
img = io.imread('./dataset/test/results/1_predict.png')
img=img.astype(np.float32)
mean2=dice_coef(img2/255, img/255)
print("Mean Dice Coeff : ",mean2)

Mean Dice Coeff :  tf.Tensor(0.9407282, shape=(), dtype=float32)


In [74]:
img = io.imread('./dataset/test/label/2.png')
img = img.astype(np.float32)
img3 = trans.resize(img,[256,256])
img = io.imread('./dataset/test/results/2_predict.png')
img = img.astype(np.float32)
mean3 = dice_coef(img3/255, img/255)
print("Mean Dice Coeff : ",mean3)

Mean Dice Coeff :  tf.Tensor(0.9360681, shape=(), dtype=float32)


In [76]:
mean_average=(mean1+mean2+mean3)/3
print(mean_average)

tf.Tensor(0.9405921, shape=(), dtype=float32)
